#### Install libraries and dependencies

In [2]:
!pip install bs4

     |████████████████████████████████| 122kB 6.4MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [3]:
# Import Libraries
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

#### Import the wikipedia page and get the raw table from the url

In [4]:
# This is the wikipedia url with the data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")

In [5]:
tables = soup.find_all('table')
# len(tables)

In [6]:
for index, table in enumerate(tables):
    if ("M3A" in str(table)):
        table_index = index
#print(table_index)

In [7]:
toronto_data = pd.DataFrame(columns=["col1", "col2", "col3", "col4", "col5", "col6", "col7", "col8", "col9"])

for row in tables[0].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        col1 = col[0].text
        col2 = col[1].text
        col3 = col[2].text
        col4 = col[3].text
        col5 = col[4].text
        col6 = col[5].text
        col7 = col[6].text
        col8 = col[7].text
        col9 = col[8].text
        toronto_data = toronto_data.append({"col1":col1, "col2":col2, "col3":col3, "col4":col4, "col5":col5, "col6":col6,"col7":col7, "col8":col8, "col9":col9}, ignore_index=True)

#toronto_data.head()

### Rework the table to be able to work with it

In [8]:
toronto_data_new = toronto_data

# Make all 9 columns into one column:

df1 = toronto_data_new["col1"]
df2 = toronto_data_new["col2"]
df3 = toronto_data_new["col3"]
df4 = toronto_data_new["col4"]
df5 = toronto_data_new["col5"]
df6 = toronto_data_new["col6"]
df7 = toronto_data_new["col7"]
df8 = toronto_data_new["col8"]
df8 = toronto_data_new["col9"]

pieces = (df1,df2,df3,df4,df5,df6,df7,df8)

data_final = pd.concat(pieces, ignore_index = True)

data_final = pd.DataFrame(data_final)
data_final.rename(columns={0:'StrData'}, inplace=True)

#data_final.head()

In [9]:
# Remove all data points where the borough is not assigned

data_final = data_final[~data_final.StrData.str.contains("Not assigned")]
#data_final.head()

In [10]:
# Extract the postal code

data_final["PostCode"] = data_final['StrData'].str[1:4]
#data_final.head()

In [11]:
# Create the Borough column

data_final["Borough"] = data_final.StrData.str.extract(r'(?P<Str>[^(]+)')
data_final["Borough"] = data_final['Borough'].str[4:len(data_final['Borough'])]

#data_final.head()

In [12]:
# Create the Neighbourhood column

data_final["Neighbourhood"] = data_final.StrData.str.replace(r'[^(]*\(|\)[^)]*', '')
#data_final.head()

toronto_dataset = data_final
toronto_dataset.drop(columns=["StrData"], inplace = True)

#toronto_dataset.head()

In [13]:
# Replace the slash with a column

toronto_dataset = toronto_dataset.replace(r' /',',', regex=True)
toronto_dataset.head()

,PostCode,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


### Dataframe obtained and now checked the no. of rows and columns

In [14]:
toronto_dataset.shape

(98, 3)

## Part II: Obtain the longitude/latitude information for each postcode in the dataset

In [15]:
# Read in the geo data from the csv file

geo_data = pd.read_csv('Geospatial_Coordinates.csv')
#geo_data.head()

In [16]:
# merge the geo_data dataset with the toronto_dataset based on the postal code
toronto_data = toronto_dataset
toronto_data.rename(columns={'PostCode': 'Postal Code'}, inplace = True)

toronto_data = pd.merge(toronto_data, geo_data, on="Postal Code")
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part III: Explore and cluster neighbourhoods in Toronto

#### We are in the Toronto Boroughs and want to go swimming, where should we go?
#### Then after the swim, we would like to grab some food to eat, so which neighbourhood should we go to? Which ones have lots of restaurants?

In [17]:
# Import libraries
import requests # library to handle requests
import random # library for random number generation
import numpy as np

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 112kB 6.4MB/s eta 0:00:01
Folium installed
Libraries imported.


In [19]:
#toronto_data.head()

In [20]:
# Select boroughs that are only in Toronto

#toronto_data.Borough.unique()

toronto_data_subset = toronto_data[toronto_data['Borough'].str.contains("Toronto")]
#toronto_data_subset.head()

In [21]:
# Here obtain the geographical coordinates for Toronto

address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada 43.6534817, -79.3839347.


In [22]:
# Make a map to visualize where the neighbourhoods in Toronto are that we're intereseted in

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# Adding markers to map
for lat, lng, neighborhood in zip(toronto_data_subset['Latitude'],  toronto_data_subset['Longitude'], toronto_data_subset['Neighbourhood']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='black',fill=True,fill_color='black',fill_opacity=0.7).add_to(map_toronto)
map_toronto

In [23]:
# Include the FourSquare Credentials

CLIENT_ID = '5FQZACKYU0DLZYLIFLOB4I1FGTILI2M3RI32RPIC054FCM53' # your Foursquare ID
CLIENT_SECRET = 'W4FCR0PLPDV2P1HT5JHYSUQCOBB0PMASJJRIXB450LU2XB3V' # your Foursquare Secret
ACCESS_TOKEN = 'ZROUUEPHT0UOZ2NNDCQDKGIGCAVWC30DGYX4C34M1SGCHMWX' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5FQZACKYU0DLZYLIFLOB4I1FGTILI2M3RI32RPIC054FCM53
CLIENT_SECRET:W4FCR0PLPDV2P1HT5JHYSUQCOBB0PMASJJRIXB450LU2XB3V


In [24]:
# Search for swimming pools
search_query = 'Pool'
radius = 500
print(search_query + ' .... OK!')

Pool .... OK!


In [25]:
# Define the url
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5FQZACKYU0DLZYLIFLOB4I1FGTILI2M3RI32RPIC054FCM53&client_secret=W4FCR0PLPDV2P1HT5JHYSUQCOBB0PMASJJRIXB450LU2XB3V&ll=43.6534817,-79.3839347&oauth_token=ZROUUEPHT0UOZ2NNDCQDKGIGCAVWC30DGYX4C34M1SGCHMWX&v=20180604&query=Pool&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
#results

In [27]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.state,location.country,location.formattedAddress,location.crossStreet,location.city,location.postalCode,location.neighborhood
0,515e0d16e4b0e99f5f4764a8,Pool,"[{'id': '4bf58dd8d48988d15e941735', 'name': 'P...",v-1617194184,False,Intercontinental,43.650975,-79.384053,"[{'label': 'display', 'lat': 43.65097492328021...",279,CA,Ontario,Canada,"[Intercontinental, Ontario, Canada]",NaN,NaN,NaN,NaN
1,51c0b4bf498e78941e4f6a9d,Swimming Pool,"[{'id': '4bf58dd8d48988d132951735', 'name': 'H...",v-1617194184,False,123 Queen St W,43.652266,-79.384815,"[{'label': 'display', 'lat': 43.652266, 'lng':...",152,CA,ON,Canada,[123 Queen St W (between Bay St & University A...,between Bay St & University Ave,Toronto,NaN,NaN
2,4bf46261cad2c9289bdc9b99,Deck 27 Pool & Fitness Facility,"[{'id': '4bf58dd8d48988d132951735', 'name': 'H...",v-1617194184,False,33 Gerrard St W,43.658170,-79.383062,"[{'label': 'display', 'lat': 43.65816982438936...",526,CA,ON,Canada,[33 Gerrard St W (in Delta Chelsea on 27th Flo...,in Delta Chelsea on 27th Floor,Toronto,NaN,NaN
3,4fe8ce26e4b08e1d007d1088,Doubletree Pool & Spa,"[{'id': '4bf58dd8d48988d15e941735', 'name': 'P...",v-1617194184,False,108 Chestnut Street,43.654580,-79.386118,"[{'label': 'display', 'lat': 43.65457985232015...",214,CA,ON,Canada,"[108 Chestnut Street (Dundas), Toronto ON M5G ...",Dundas,Toronto,M5G 1R3,NaN
4,5133f6d0e4b08f94a76512ab,The Pool at Shangri-La,"[{'id': '4bf58dd8d48988d132951735', 'name': 'H...",v-1617194184,False,188 University Ave,43.649251,-79.385598,"[{'label': 'display', 'lat': 43.64925135414941...",489,CA,ON,Canada,"[188 University Ave (Adelaide St. W), Toronto ...",Adelaide St. W,Toronto,M5H 0A3,Financial District


In [28]:
# Filter the dataframe to only keep points of interest

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,state,country,formattedAddress,crossStreet,city,postalCode,neighborhood,id
0,Pool,Pool,Intercontinental,43.650975,-79.384053,"[{'label': 'display', 'lat': 43.65097492328021...",279,CA,Ontario,Canada,"[Intercontinental, Ontario, Canada]",NaN,NaN,NaN,NaN,515e0d16e4b0e99f5f4764a8
1,Swimming Pool,Hotel Pool,123 Queen St W,43.652266,-79.384815,"[{'label': 'display', 'lat': 43.652266, 'lng':...",152,CA,ON,Canada,[123 Queen St W (between Bay St & University A...,between Bay St & University Ave,Toronto,NaN,NaN,51c0b4bf498e78941e4f6a9d
2,Deck 27 Pool & Fitness Facility,Hotel Pool,33 Gerrard St W,43.658170,-79.383062,"[{'label': 'display', 'lat': 43.65816982438936...",526,CA,ON,Canada,[33 Gerrard St W (in Delta Chelsea on 27th Flo...,in Delta Chelsea on 27th Floor,Toronto,NaN,NaN,4bf46261cad2c9289bdc9b99
3,Doubletree Pool & Spa,Pool,108 Chestnut Street,43.654580,-79.386118,"[{'label': 'display', 'lat': 43.65457985232015...",214,CA,ON,Canada,"[108 Chestnut Street (Dundas), Toronto ON M5G ...",Dundas,Toronto,M5G 1R3,NaN,4fe8ce26e4b08e1d007d1088
4,The Pool at Shangri-La,Hotel Pool,188 University Ave,43.649251,-79.385598,"[{'label': 'display', 'lat': 43.64925135414941...",489,CA,ON,Canada,"[188 University Ave (Adelaide St. W), Toronto ...",Adelaide St. W,Toronto,M5H 0A3,Financial District,5133f6d0e4b08f94a76512ab


In [29]:
# Now that we have the list of pools in Toronto, I would like to see their names and categories on the map as well as the neighbourhoods

dataframe_filtered["name_categories"] = dataframe_filtered["name"] + "+" + dataframe_filtered["categories"]


pools_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Toronto

# Adding neighbourhood name markers to map
for lat, lng, neighborhood in zip(toronto_data_subset['Latitude'],  toronto_data_subset['Longitude'], toronto_data_subset['Neighbourhood']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='black',fill=True,fill_color='black',fill_opacity=0.7).add_to(pools_map)


# add pools to the map
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name_categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(pools_map)

# display map
pools_map

### We wanted to go to an athletic pool (not a spa), so we have found a pool of interest: the Harrison Pool + Gym/Fitness Center and we wanted to know the rating of it

In [30]:
#dataframe_filtered.head()
dataframe_filtered[dataframe_filtered['name_categories'] == "Harrison Pool+Gym / Fitness Center"]

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,state,country,formattedAddress,crossStreet,city,postalCode,neighborhood,id,name_categories
11,Harrison Pool,Gym / Fitness Center,15 Stephanie St,43.651291,-79.390938,"[{'label': 'display', 'lat': 43.651291, 'lng':...",614,CA,ON,Canada,"[15 Stephanie St (McCaul St), Toronto ON, Canada]",McCaul St,Toronto,NaN,NaN,4da5005a1e72a3244fa612a2,Harrison Pool+Gym / Fitness Center


In [31]:
# Examine the ratings for the Harrison pool

venue_id = '4da5005a1e72a3244fa612a2' # ID of Harrison's Pool
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4da5005a1e72a3244fa612a2?client_id=5FQZACKYU0DLZYLIFLOB4I1FGTILI2M3RI32RPIC054FCM53&client_secret=W4FCR0PLPDV2P1HT5JHYSUQCOBB0PMASJJRIXB450LU2XB3V&oauth_token=ZROUUEPHT0UOZ2NNDCQDKGIGCAVWC30DGYX4C34M1SGCHMWX&v=20180604'

In [32]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4da5005a1e72a3244fa612a2',
 'name': 'Harrison Pool',
 'contact': {},
 'location': {'address': '15 Stephanie St',
  'crossStreet': 'McCaul St',
  'lat': 43.651291,
  'lng': -79.390938,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.651291,
    'lng': -79.390938}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['15 Stephanie St (McCaul St)', 'Toronto ON', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/harrison-pool/4da5005a1e72a3244fa612a2',
 'categories': [{'id': '4bf58dd8d48988d175941735',
   'name': 'Gym / Fitness Center',
   'pluralName': 'Gyms or Fitness Centers',
   'shortName': 'Gym / Fitness',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'likes': {'count': 0, 'groups': []},
 'like': False,
 'dislike': False,
 'ok': False,
 'allowMenuUrlEdit': True,
 'beenHere': {'count': 0,
  'unco

In [33]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


### We realized that it hasn't been rated yet... we feel adventurous so we will go there all the same and can leave the first review!!

### Now that we have worked up an appetite, we would like to find neighbourhoods with some good food!

In [34]:
toronto_data_subset.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923


In [35]:
# Here obtain the geographical coordinates for Toronto

address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada 43.6534817, -79.3839347.


In [36]:
# We want to get a list of venues for each Toronto neighbourhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(names=toronto_data_subset['Neighbourhood'],
                                   latitudes=toronto_data_subset['Latitude'],
                                   longitudes=toronto_data_subset['Longitude']
                                  )

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [38]:
print(toronto_venues.shape)
toronto_venues.head()

(1571, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth East,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park


In [41]:
# Now we do one-hot encoding to see which sort of venue is present in which neighbourhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()
#toronto_onehot.shape

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Here we average the onehot encoding frequencies per neighbourhood

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0,0.015625
4,Christie,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [45]:
toronto_grouped.shape

(38, 232)

In [50]:
# This is the function for getting the venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
# Here we want to see what are the top venues per neighbourhood, so let's do the top 5

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop,Pharmacy,Restaurant,Seafood Restaurant,Farmers Market,Juice Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Stadium,Bar,Intersection,Bakery,Restaurant,Climbing Gym,Furniture / Home Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Salad Place,Burger Joint,Thai Restaurant,Japanese Restaurant,Department Store
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Baby Store,Candy Store,Athletics & Sports,Restaurant,Coffee Shop


In [53]:
# Now we cluster the neighbourhoods

from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [62]:
toronto_data.rename(columns={"Neighbourhood":"Neighborhood"}, inplace = True)
#toronto_data.head()

In [74]:
#neighborhoods_venues_sorted.tail()
toronto_data.head()
#toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [82]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Merge neighborhoods_venues_sorted with toronto_data on Neighbourhood

toronto = neighborhoods_venues_sorted.merge(toronto_data, on='Neighborhood')

In [92]:
toronto.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
0,0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Cheese Shop,Pharmacy,Restaurant,Seafood Restaurant,Farmers Market,Juice Bar,M5E,Downtown Toronto,43.644771,-79.373306
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Stadium,Bar,Intersection,Bakery,Restaurant,Climbing Gym,Furniture / Home Store,M6K,West Toronto,43.636847,-79.428191
2,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique,M5V,Downtown Toronto,43.628947,-79.394420
3,0,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Salad Place,Burger Joint,Thai Restaurant,Japanese Restaurant,Department Store,M5G,Downtown Toronto,43.657952,-79.387383
4,0,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Baby Store,Candy Store,Athletics & Sports,Restaurant,Coffee Shop,M6G,Downtown Toronto,43.669542,-79.422564


In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood'], toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="red",
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [91]:
toronto.loc[toronto['1st Most Common Venue'] == "Restaurant"]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude


In [ ]:
# There are no restuarants in the 1st most common venue, so let's try the 2nd

In [93]:
toronto.loc[toronto['2nd Most Common Venue'] == "Restaurant"]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
6,0,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Cocktail Bar,M5L,Downtown Toronto,43.648198,-79.379817
20,0,"Little Portugal, Trinity",Bar,Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Café,Coffee Shop,Men's Store,Asian Restaurant,Yoga Studio,Cupcake Shop,M6J,West Toronto,43.647927,-79.419750


#### Looks like the Commerce Court, Victoria Hotel and the Little Portugal, Trinity neighbourhoods are the most likely to have lots of restaurants to choose from. We are hungry and in the mood for Mexican food. The former looks more inviting as it has more coffee shops than bars and so let's check it out

In [103]:
# Let's get a list of restaurants that are in these neighbourhoods - we are in the mood for Mexican, so let's check that out

# Search for restaurants
search_query = 'Mexican'
radius = 500
print(search_query + ' .... OK!')

Mexican .... OK!


In [104]:
# The geolocation of the commerce court, victoria hotel neighbourhood is:
latitude = 43.648198
longitude = -79.379817

In [105]:
# Define the url
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5FQZACKYU0DLZYLIFLOB4I1FGTILI2M3RI32RPIC054FCM53&client_secret=W4FCR0PLPDV2P1HT5JHYSUQCOBB0PMASJJRIXB450LU2XB3V&ll=43.648198,-79.379817&oauth_token=ZROUUEPHT0UOZ2NNDCQDKGIGCAVWC30DGYX4C34M1SGCHMWX&v=20180604&query=Mexican&radius=500&limit=100'

In [106]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60647a749a07886de2403ce8'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4f7d9a0ee4b01f9f43bd0603',
    'name': 'Mexican Consulate',
    'location': {'address': '11 King St',
     'lat': 43.64894155562399,
     'lng': -79.37867590887817,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64894155562399,
       'lng': -79.37867590887817}],
     'distance': 123,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['11 King St', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d12c951735',
      'name': 'Embassy / Consulate',
      'pluralName': 'Embassies / Consulates',
      'shortName': 'Embassy',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/embassy_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1617197684',
    'hasPerk': False},
   {'id': '4

In [116]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe_rest = json_normalize(venues)
dataframe_rest.head()

dataframe_rest.rename(columns={"location.lat":"lat", "location.lng":"lng"}, inplace = True)
dataframe_rest.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,lat,lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4f7d9a0ee4b01f9f43bd0603,Mexican Consulate,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1617197684,False,11 King St,43.648942,-79.378676,"[{'label': 'display', 'lat': 43.64894155562399...",123,CA,Toronto,ON,Canada,"[11 King St, Toronto ON, Canada]",NaN,NaN
1,4ae623e3f964a52002a521e3,Quesada Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1617197684,False,92 King Street E,43.649939,-79.374496,"[{'label': 'display', 'lat': 43.64993863703611...",470,CA,Toronto,ON,Canada,"[92 King Street E (at Church St), Toronto ON M...",at Church St,M5C 2V8
2,4bdef9836198c9b6de8115ff,Mucho Burrito Fresh Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1617197684,False,333 Bay St.,43.650661,-79.379401,"[{'label': 'display', 'lat': 43.65066054666582...",276,CA,Toronto,ON,Canada,"[333 Bay St. (Bay Adelaide Centre), Toronto ON...",Bay Adelaide Centre,M5H 2R2
3,4d76c9eec95f5941ca6e2321,Chimichanga Mexican Grill,[],v-1617197684,False,70 Yorkville Ave,43.651772,-79.379482,"[{'label': 'display', 'lat': 43.65177154540889...",398,CA,Toronto,ON,Canada,"[70 Yorkville Ave (Yorkville/Bel Air), Toronto...",Yorkville/Bel Air,M5R 1B9
4,4f15c767e4b0380207013250,Chipotle Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1617197684,False,"123 Front St W, Unit 103",43.644890,-79.383050,"[{'label': 'display', 'lat': 43.64489, 'lng': ...",451,CA,Toronto,ON,Canada,"[123 Front St W, Unit 103 (at University Ave),...",at University Ave,M5J 2M2


In [108]:
# Filter the dataframe to only keep points of interest

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_rest = dataframe_rest.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_rest['categories'] = dataframe_rest.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_rest.columns = [column.split('.')[-1] for column in dataframe_rest.columns]

dataframe_rest.head()

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['location.neighborhood'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [122]:
poollat = 43.651291
poollong = -79.390938


food_map = folium.Map(location=[poollat, poollong], zoom_start=15) # generate map centred around the pool

# add the swimming pool to the map

folium.CircleMarker(
    [poollat, poollong],
    radius=10,
    popup='HarrisonPool',
    fill=True,
    color='black',
    fill_color='blue',
    fill_opacity=0.6
    ).add_to(food_map)

#add Mexican restaurants to the map
for lat, lng, label in zip(dataframe_rest.lat, dataframe_rest.lng, dataframe_rest.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='black',
        fill_color='black',
        fill_opacity=0.6
        ).add_to(food_map)

# display map
food_map

### There are 4 Mexican restaurants around the pool, let's map where they are and find the closest to the pool (we're hungry)

### Which one has the best rating?

In [197]:
# Examine the ratings for the Harrison pool

# '4f0240df6da1cd035b2227eb' # ID of Mucho Burrito
# '59304fea4186863fd6a112bb' # ID of Chipotle
# '4b880141f964a5205dd831e3' # ID of Quesadea Burritos
# '4afda2faf964a5205d2922e3' # ID of Milagro

venue_id = '4afda2faf964a5205d2922e3' # ID of Milagro
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4afda2faf964a5205d2922e3?client_id=5FQZACKYU0DLZYLIFLOB4I1FGTILI2M3RI32RPIC054FCM53&client_secret=W4FCR0PLPDV2P1HT5JHYSUQCOBB0PMASJJRIXB450LU2XB3V&oauth_token=ZROUUEPHT0UOZ2NNDCQDKGIGCAVWC30DGYX4C34M1SGCHMWX&v=20180604'

In [198]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4afda2faf964a5205d2922e3',
 'name': 'Milagro Cantina Mexicana',
 'contact': {'phone': '+14168502855',
  'formattedPhone': '+1 416-850-2855',
  'twitter': 'milagro_cantina',
  'facebook': '489583111091514',
  'facebookUsername': 'MilagroTO',
  'facebookName': 'Milagro Cantina'},
 'location': {'address': '5 Mercer St',
  'crossStreet': 'at John',
  'lat': 43.645097,
  'lng': -79.389337,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.645097,
    'lng': -79.389337}],
  'postalCode': 'm5v1h2',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['5 Mercer St (at John)',
   'Toronto ON m5v1h2',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/milagro-cantina-mexicana/4afda2faf964a5205d2922e3',
 'categories': [{'id': '4bf58dd8d48988d1c1941735',
   'name': 'Mexican Restaurant',
   'pluralName': 'Mexican Restaurants',
   'shortName': 'Mexican',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
   

In [199]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.8


### Results:
### Mucho Burrito = 6.3
### Chipotle = 7.4
### Quesadea Burritos = 5.5
### Milagro = 6.8

## We go to Chipotle as it's the closest and highest rated
